In [1]:
%load_ext autoreload
%autoreload 2
import ase
from ase.visualize import view
from ase.build import molecule
from ase.io import Trajectory, write, read
from ase.units import mol # Avogadro Constant
# calculator
import pfp_api_client
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
# Molecular Dynamics
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary
import flare
from flare.ase.nosehoover import NoseHoover
from ase.units import kB, fs, Pascal

import numpy as np

def vis(atoms):
    v = view(atoms, viewer='ngl')
    v.view.add_representation("ball+stick")
    display(v)

#estimator = Estimator(calc_mode='CRYSTAL_PLUS_D3')
estimator = Estimator(calc_mode=EstimatorCalcMode.CRYSTAL_PLUS_D3)
calculator = ASECalculator(estimator)

from ase.constraints import FixAtoms

print(f"pfp_api_client: {pfp_api_client.__version__}")
print(f"Estimator model version: {estimator.model_version}")
print("available versions: ", estimator.available_models)

pfp_api_client: 1.4.0
Estimator model version: v3.0.0
available versions:  ['latest', 'v0.0.0', 'v1.0.0', 'v1.1.0', 'v2.0.0', 'v3.0.0']


In [2]:
# for I/O

E_FIELD=0.3
INIT_dataf="H2O_HCl_NVT_0.1ns_final.traj"
INIT_read_index=":"

OUT_traj="H2O_HCl_E"+ str(E_FIELD) + "_0.2ns.traj"
OUT_log="H2O_HCl_E"+ str(E_FIELD) + "_0.2ns.log"
OUT_xyz="H2O_HCl_E"+ str(E_FIELD) + "_0.2ns_charge.xyz"

OUT_traj_itv=100
OUT_log_itv=50

##

data_path = INIT_dataf
print(data_path)
initial_structure = read(data_path)


H2O_HCl_NVT_0.1ns_final.traj


In [3]:
from langevin_ExternalField import Langevin

pos = initial_structure

T = 300.0
#P = 100000*Pascal
dt = 0.25*fs

pos.set_calculator(calculator)
efield = np.array([0,E_FIELD,0])
dyn = Langevin(atoms=pos, timestep=dt, 
               temperature_K = T, friction=0.01, efield=efield, trajectory = OUT_traj, loginterval=OUT_traj_itv)

#dyn = NPT(pos, dt, temperature_K = T, externalstress = P, ttime = tdamp, pfactor = pdamp, trajectory = OUT_traj, loginterval=100)

Header= "step etotal T e_pot stress"
print("NVT Simulation")
print(Header)

weight = np.sum(pos.get_masses())*1.673e-27

file = open(OUT_log, "w")
file.write(Header + "\n")
def print_dyn():
    line = f"{dyn.get_number_of_steps(): >3} {pos.get_total_energy():.6f} {pos.get_temperature():.6f} {pos.get_potential_energy():.6f} {-pos.get_stress(include_ideal_gas=True)[0:3].sum()/(3.0*Pascal):.6f}"    
    file.write(line+"\n")
    print(line, end = "\r")
    
    
dyn.attach(print_dyn, interval=OUT_log_itv)

#steps = 800000
steps = 100
dyn.run(steps)


file.close()

NVT Simulation
step etotal T e_pot stress


In [4]:
a = read(OUT_traj, index=':')

def savexyz(traj, filename='pos.xyz'):
    index = 0
    for atoms in traj:
        atoms.wrap()
        atoms.set_calculator(calculator)
        aa=atoms.get_charges()
        #aa = aa.reshape([-1])
        #print(aa)
        #atoms.set_initial_charges( aa )
        write(filename, atoms, append = True)
        index = index + 1

        
savexyz(traj=a, filename=OUT_xyz)
#vis(a)